<a href="https://colab.research.google.com/github/Noertri/scraper-nusabali/blob/main/scraper_nusabali.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Menginstall package

1. upload file requirements.txt
2. jalankan sel dibawah

In [ ]:
!pip install --upgrade -r requirements.txt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.0/143.0 kB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 293.8/293.8 kB 6.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.4/75.4 kB 7.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 250.0/250.0 kB 6.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 74.5/74.5 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: openpyxl
    Found existing installation: openpyxl 3.0.10
    Uninstalling openpyxl-3.0.10:
      Successfully uninstalled openpyxl-3.0.10
  Attempting uninstall: beautifulsoup4
    Found existing installation: beautifulsoup4 4.11.2
    Uninstalling beautifulsoup4-4.11.2:
      Successfully uninstalled beautifulsoup4-4.11.2


# Program

In [3]:
import httpx
from bs4 import BeautifulSoup
from dataclasses import dataclass, asdict
import pandas as pd
from urllib import parse
import dateparser
from datetime import datetime


client = httpx.Client(follow_redirects=True, timeout=5.)
main_url = "https://www.nusabali.com"
results = []


@dataclass
class Result:
    tanggal: str = ""
    judul: str = ""
    url: str = ""


def request_main_page(keyword):
    page_url = parse.urljoin(main_url, "search")

    params = {
        'keyword': f'{keyword}',
    }

    query_str = parse.urlencode(params)

    referer_url = parse.urlunsplit(("https", "www.nusabali.com", "search", query_str, ""))

    headers = {
        'authority': 'www.nusabali.com',
        'accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,*/*;q=0.8,application/signed-exchange;v=b3;q=0.7',
        'accept-language': 'id-ID,id;q=0.9,en-US;q=0.8,en;q=0.7,ms;q=0.6,ja;q=0.5',
        'dnt': '1',
        'referer': f'{referer_url}',
        'sec-ch-ua': '"Not/A)Brand";v="99", "Google Chrome";v="115", "Chromium";v="115"',
        'sec-ch-ua-mobile': '?0',
        'sec-ch-ua-platform': '"Windows"',
        'sec-fetch-dest': 'document',
        'sec-fetch-mode': 'navigate',
        'sec-fetch-site': 'same-origin',
        'sec-fetch-user': '?1',
        'upgrade-insecure-requests': '1',
        'user-agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/115.0.0.0 Safari/537.36',
    }


    response = client.get(page_url, params=params, headers=headers)

    if response.status_code < 400:
        return response


# scrape halaman awal
r = request_main_page(keyword="bali")
soup = BeautifulSoup(r.text, "html.parser")
article_list = soup.select("div#article-list>div.row")
if article_list:
    for article in article_list:
        h5_tag = article.select_one("div.col-xs-8.col-md-8 h5")
        span_tag = article.select_one("div.col-xs-12.entry-metas>span>span")
        article_title = h5_tag.get_text(strip=True, separator=" ") if h5_tag is not None else ""
        article_url = a_tag.get("href", "") if (a_tag := h5_tag.select_one("a")) is not None else ""
        article_date = span_tag.get_text(strip=True, separator=" ") if span_tag is not None else ""
        article_date = datetime.strptime(article_date, "%d %b %Y %H:%M")

        result = Result(tanggal=article_date.strftime("%d %B %Y %H:%M"), judul=article_title, url=article_url)
        results.append(asdict(result))



In [15]:
df = pd.DataFrame(results)

df

,tanggal,judul,url
0,01 August 2023 10:50,Pevoli Indonesia Dominasi Gelar Individu SEA V...,https://www.nusabali.com/berita/147388/pevoli-...
1,01 August 2023 10:43,Eden Hazard Tanpa Klub dan Terancam Pensiun,https://www.nusabali.com/berita/147362/eden-ha...
2,01 August 2023 10:41,Kasus Adu Jotos antar Kader Golkar Klungkung B...,https://www.nusabali.com/berita/147369/kasus-a...
3,01 August 2023 10:35,Karangasem Tidak Punya Pengawas TK,https://www.nusabali.com/berita/147354/karanga...
4,01 August 2023 10:33,150 Produk Jembrana Tersertifikasi Halal,https://www.nusabali.com/berita/147348/150-pro...
5,01 August 2023 10:27,Gubernur Koster Sambut Kedatangan FC Barcelona...,https://www.nusabali.com/berita/147352/gubernu...
6,01 August 2023 09:48,Megawati Gembleng Ketua DPD dan DPC Partai,https://www.nusabali.com/berita/147367/megawat...
7,01 August 2023 09:02,"Dijerat Pasal Berlapis, Langsung Disel",https://www.nusabali.com/berita/147370/dijerat...
8,01 August 2023 08:57,Putri Koster Gelar Ramah Tamah dengan Teater M...,https://www.nusabali.com/berita/147360/putri-k...
9,01 August 2023 08:55,Pilkel 14 Desa Tabanan Nihil Gugatan,https://www.nusabali.com/berita/147347/pilkel-...
